# Implementation of matrix operations in C

## Conventions for storing matrices in memory

The C convention is that matrices are stored in memory row-by-row.  In order to store an $M\times N$ matrix with real entries, where we assume that each entry is represented as a double precision floating point number, we need to allocate at least ```M*N*sizeof(double)``` bytes of memory; for example
```C
int M, N;
double *A;
/* define M and N */
/* allocate memory for A */
A = (double*)malloc(M*N*sizeof(double));
/* do stuff with A */
/* deallocate memory */
free(A);
```
To access the element $A_{i,j}$ of the matrix defined above we write ```A[i*N + j]```, where $0\leq i < N$, $0\leq j < M$.
Sometimes it will be convenient to look at submatrices of larger matrices.  In this case, we need to distinguish between the number of columns in the submatrix ($N$ in our notation) and the spacing in memory between the elements $A_{i,j}$ and $A_{i+1,j}$ (aka stride).  In order to allow this option, many numerical codes in addition to the dimensions $M$, $N$ of the matrix require us to provide an additional integer argument defining the stride, historically called ```LDA```.  In this notation, to access the element $A_{i,j}$ we write ```A[i*LDA + j]```.

## Matrix addition

Let us write a function computing the sum of two matrices, $A$ and $B$ of size $M\times N$, and store the result in the matrix $C$ of the same size.  The strides of the matrices are given as ```LDA```, ```LDB```, and ```LDC```.

In [13]:
%%writefile matrix_add.c

void matrix_add(int M, int N,
                double *A, int LDA,
                double *B, int LDB,
                double *C, int LDC)
{
    // Variable declarations.
    int i, j;

    // Loop over matrix rows
    for (i = 0; i < M; ++i)
        // Loop over columns
        for (j = 0; j < N; ++j)
            C[i*LDC + j] = A[i*LDA + j] + B[i*LDB + j];
}

Overwriting matrix_add.c


Let us compile this function and produce a shared library matrix_add:

In [9]:
!!gcc -Wall -shared -Wl,-install_name,matrix_add \
    -o matrix_add.so \
    -fPIC matrix_add.c

[]

We can now load this library from Python.

In [14]:
# Load the shared library
import ctypes
lib = ctypes.CDLL('./matrix_add.so')
matrix_add = lib.matrix_add
from numpy.ctypeslib import ndpointer
# Define the types of the output and arguments of
# this function.
matrix_add.restype = None
matrix_add.argtypes = [ctypes.c_int, #M
                       ctypes.c_int, #N
                       ndpointer(ctypes.c_double), ctypes.c_int, #A, LDA
                       ndpointer(ctypes.c_double), ctypes.c_int, #B, LDB
                       ndpointer(ctypes.c_double), ctypes.c_int  #C, LDC
                       ]

Here is a very simple test; we add two $2\times 3$ matrices.

In [11]:
import numpy as np
# define a couple of test arrays
A = np.array([[1,2,3],[4,5,6]],dtype=np.double)
B = np.array([[7,8,9],[10,11,12]],dtype=np.double)
M,N = A.shape[0],A.shape[1]
# and a zero output array
C = np.zeros_like(A)
print("C before")
print(C)
# call add function; use N as the stride for all matrices
matrix_add(M,N,A,N,B,N,C,N)
print("C after")
print(C)

C before
[[0. 0. 0.]
 [0. 0. 0.]]
C after
[[ 8. 10. 12.]
 [14. 16. 18.]]


Let us now repeat the same process, but we shall view $A$, $B$, and $C$ as $2x2$ submatrices of the original arrays, while supplying the stride of $3$:

In [15]:
M,N = A.shape[0],2
LDA = A.shape[1]
# zero out the output array
C[:] = 0
print("C before")
print(C)
# call add function; use N as the stride for all matrices
matrix_add(M,N,A,LDA,B,LDA,C,LDA)
print("C after")
print(C)

C before
[[0. 0. 0.]
 [0. 0. 0.]]
C after
[[ 8. 10.  0.]
 [14. 16.  0.]]


## Naive matrix multiplication

Let us now write a textbook version of the matrix multiplication routine.  Our goal is to compute $C=AB$, and we assume that the size of $A$ is $M\times N$ and the size of $B$ is $N\times K$. Consequently $C$ has the size $M\times K$.  The strides are also provided as in the previous example of matrix addition.

In [21]:
%%writefile matrix_mul.c

void matrix_mul(int M, int N, int K,
                double *A, int LDA,
                double *B, int LDB,
                double *C, int LDC)
{
    // Variable declarations.
    int i, j, l;
    double Cij;

    // Loop over rows of C
    for (i = 0; i < M; ++i)
        // Loop over columns of C
        for (j = 0; j < K; ++j)
        {
          // zero out the Cij
          Cij = 0.0;
          // compute the summation: innermost loop
          for (l = 0; l < N; ++l)
            Cij += A[i*LDA + l] * B[l*LDB + j];
          // put the result into the matrix C
          C[i*LDC + j] = Cij;
        }
}

Overwriting matrix_mul.c


In [22]:
!!gcc -Wall -shared -Wl,-install_name,matrix_mul \
    -o matrix_mul.so \
    -fPIC matrix_mul.c

[]

In [23]:
# Load the shared library
import ctypes
lib = ctypes.CDLL('./matrix_mul.so')
matrix_mul = lib.matrix_mul
from numpy.ctypeslib import ndpointer
# Define the types of the output and arguments of
# this function.
matrix_mul.restype = None
matrix_mul.argtypes = [ctypes.c_int, #M
                       ctypes.c_int, #N
                       ctypes.c_int, #K                       
                       ndpointer(ctypes.c_double), ctypes.c_int, #A, LDA
                       ndpointer(ctypes.c_double), ctypes.c_int, #B, LDB
                       ndpointer(ctypes.c_double), ctypes.c_int  #C, LDC
                       ]

Let us test the multiplication routine on some simple matrices.

In [26]:
# define a couple of test arrays
A = np.array([[1,2,3],[4,5,6]],dtype=np.double)
B = np.array([[7,8,9,10],[11,12,13,14],[15,16,17,18]],dtype=np.double)
M,N,K = A.shape[0],A.shape[1],B.shape[1]
C = np.zeros((M,K),dtype=np.double)
print("C before")
print(C)
# call add function; use N as the stride for all matrices
matrix_mul(M,N,K,A,N,B,K,C,K)
print("C after")
print(C)

C before
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
C after
[[ 74.  80.  86.  92.]
 [173. 188. 203. 218.]]


## Strassen's algorithm for matrix multiplication

We now implement a more efficient (for larger matrices) multiplication algorithm.

In [67]:
%%writefile matrix_strassen.c

#include <stdlib.h>

/* Helper function 1;
   compute C = A + s*B */
void matrix_sadd(int nrow, int ncol,
                 const double* A, int lda, double s,
                 const double* B, int ldb,
                 double* C, int ldc)
{
  int i,j;
  for (i=0; i<nrow; ++i)
    for (j=0; j<ncol; ++j)
      C[i*ldc + j] = A[i*lda + j] + s*B[i*ldb + j];
}

/* Helper function 2;
   C = A*B, naive algorithm */
void matrix_mul(int nrowA, int ncolA, int ncolB,
                const double* A, int lda, const double* B, int ldb,
                double* C, int ldc)
{
  int i,j,k;
  double Cij;
  for (i=0; i<nrowA; ++i)
    for (j=0; j<ncolB; ++j)
      {
        Cij = 0;
        for (k=0; k<ncolA; ++k)
          Cij += A[i*lda + k]*B[k*ldb + j];
        C[i*ldc+j] = Cij;
      }
}


#define N_MIN 16

/* Main function;  
   C = A*B, Strassen algorithm. */
/* All matrices are assumed to be NxN */
void matrix_strassen(int N,
                  const double* A, int lda, const double* B, int ldb,
                  double* C, int ldc)
{
  double *Atmp, *Btmp, *M1, *M2, *M3, *M4, *M5, *M6, *M7;
  int n;

  if ((N <= N_MIN) || (N % 2))
    /* If N is odd or too small, we revert to the naive algorithm */
    matrix_mul (N,N,N,A,lda,B,ldb,C,ldc);
  else
  {
    /* N is even */
    n = N/2;
    Atmp = (double*)malloc(n*n*sizeof(double));
    Btmp = (double*)malloc(n*n*sizeof(double));
    M1   = (double*)malloc(n*n*sizeof(double));
    M2   = (double*)malloc(n*n*sizeof(double));
    M3   = (double*)malloc(n*n*sizeof(double));
    M4   = (double*)malloc(n*n*sizeof(double));
    M5   = (double*)malloc(n*n*sizeof(double));
    M6   = (double*)malloc(n*n*sizeof(double));
    M7   = (double*)malloc(n*n*sizeof(double));

    /* M1 = (A11 + A22)*(B11+B22) */
    matrix_sadd(n,n,A,lda,1.0,A+n*lda+n,lda,Atmp,n);
    matrix_sadd(n,n,B,ldb,1.0,B+n*ldb+n,ldb,Btmp,n);
    matrix_strassen(n,Atmp,n,Btmp,n,M1,n);
    /* M2 = (A21 + A22)*B11 */
    matrix_sadd(n,n,A+n*lda,lda,1.0,A+n*lda+n,lda,Atmp,n);
    matrix_strassen(n,Atmp,n,B,ldb,M2,n);
    /* M3 = A11*(B12-B22) */
    matrix_sadd(n,n,B+n,ldb,-1.0,B+n*ldb+n,ldb,Btmp,n);
    matrix_strassen(n,A,lda,Btmp,n,M3,n);
    /* M4 = A22*(B21-B11) */
    matrix_sadd(n,n,B+n*ldb,ldb,-1.0,B,ldb,Btmp,n);
    matrix_strassen(n,A+n*lda+n,lda,Btmp,n,M4,n);
    /* M5 = (A11 + A12)*B22 */
    matrix_sadd(n,n,A,lda,1.0,A+n,lda,Atmp,n);
    matrix_strassen(n,Atmp,n,B+n*ldb+n,ldb,M5,n);
    /* M6 = (A21 - A11)*(B11+B12) */
    matrix_sadd(n,n,A+n*lda,lda,-1.0,A,lda,Atmp,n);
    matrix_sadd(n,n,B,ldb,1.0,B+n,ldb,Btmp,n);
    matrix_strassen(n,Atmp,n,Btmp,n,M6,n);
    /* M7 = (A12 - A22)*(B21+B22) */
    matrix_sadd(n,n,A+n,lda,-1.0,A+n*lda+n,lda,Atmp,n);
    matrix_sadd(n,n,B+n*ldb,ldb,1.0,B+n*ldb+n,ldb,Btmp,n);
    matrix_strassen(n,Atmp,n,Btmp,n,M7,n);
    /* C12 = M3+M5 */
    matrix_sadd(n,n,M3,n,1.0,M5,n,C+n,ldc);
    /* C21 = M2+M4 */
    matrix_sadd(n,n,M2,n,1.0,M4,n,C+n*ldc,ldc);
    /* C11 = M1+M4-M5+M7 */
    matrix_sadd(n,n,M1,n,1.0,M4,n,Atmp,n);
    matrix_sadd(n,n,M7,n,-1.0,M5,n,Btmp,n);
    matrix_sadd(n,n,Atmp,n,1.0,Btmp,n,C,ldc);
    /* C22 = M1-M2+M3+M6 */
    matrix_sadd(n,n,M1,n,-1.0,M2,n,Atmp,n);
    matrix_sadd(n,n,M3,n,1.0,M6,n,Btmp,n);
    matrix_sadd(n,n,Atmp,n,1.0,Btmp,n,C+n*ldc+n,ldc);
    /* done */
    free(M7); free(M6); free(M5); free(M4);
    free(M3); free(M2); free(M1); free(Btmp); free(Atmp);
  }
}

Overwriting matrix_strassen.c


In [68]:
!!gcc -Wall -shared -Wl,-install_name,matrix_strassen \
    -o matrix_strassen.so \
    -fPIC matrix_strassen.c

[]

In [69]:
# Load the shared library
import ctypes
lib = ctypes.CDLL('./matrix_strassen.so')
matrix_strassen = lib.matrix_strassen
from numpy.ctypeslib import ndpointer
# Define the types of the output and arguments of
# this function.
matrix_strassen.restype = None
matrix_strassen.argtypes = [ctypes.c_int, #N
                       ndpointer(ctypes.c_double), ctypes.c_int, #A, LDA
                       ndpointer(ctypes.c_double), ctypes.c_int, #B, LDB
                       ndpointer(ctypes.c_double), ctypes.c_int  #C, LDC
                       ]

We now generate a couple of larger matrices, and test the naive algorithm against Strassen's

In [73]:
from timeit import default_timer as timer

N = 1024
A = np.random.rand(N,N)
B = np.random.rand(N,N)
C1= np.zeros_like(A)
C2= np.zeros_like(A)

start = timer()
matrix_mul(N,N,N,A,N,B,N,C1,N)
end   = timer()
print("Naive algorithm; elapsed = %fs" % (end-start))

start = timer()
matrix_strassen(N,A,N,B,N,C2,N)
end   = timer()
print("Strassen's algorithm; elapsed = %fs" % (end-start))

relative_error = np.linalg.norm(C1-C2,'fro')/np.linalg.norm(C1,'fro')
print("Difference between two matrices (due to round-offs) = %e" % relative_error)

Naive algorithm; elapsed = 7.710106s
Strassen's algorithm; elapsed = 2.124149s
Difference between two matrices (due to round-offs) = 3.457633e-15
